# 神经网络模型：

In [26]:
import torch

class Para:
    # tensor_board_log_dir = 'runs/exp0'
    feature_column_start_name = 'ep_ratio_ttm'
    feature_column_end_name = 'BR'

    # 模型设置
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    classification = 2 # 2, 3

    # 权重
    cross_weight = list()
    if classification == 3:
        cross_weight = [1.0, 1.0 ,1.0]
    elif classification == 2:
        cross_weight = [1.0, 1.0]
    elif classification == 5:
        cross_weight = [1.0, 1.0, 1.0, 1.0, 1.0]

    batch_size = 16
    lr = 1e-3
    drop = 0.5
    epochs = 100

    # 数据集设置
    month_in_sample = range(0, 1)
    # month_test = range(36, 48)

    percent_cv = 0.1 # 10% cross validation

    data_path = 'data/wly_space_1d_rate_20d_17-21_pre'


    seed = 2022
    torch.manual_seed(seed)

    info_str0 = data_path[5:8]+'_'+'c'+str(classification)+'_s'+str(percent_cv)
    info_str1 = '_b'+str(batch_size)+'_lr'+str(lr)+'_d'+str(drop)+'_e'+str(epochs)
    info_str = info_str0 + info_str1

    save_model_path = 'models/'+'model_'+info_str+'.pth'

para = Para()
print(para.info_str)
print(para.save_model_path)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    print(torch.cuda.current_device())

wly_c2_s0.1_b16_lr0.001_d0.5_e100
models/model_wly_c2_s0.1_b16_lr0.001_d0.5_e100.pth
True
1
GeForce MX150
0


### 构建训练集

In [27]:
import numpy as np
import pandas as pd

data_in_sample = None
for i_month in para.month_in_sample:
    file_name = para.data_path + '/' + str(i_month) + '.csv'
    data_curr_month = pd.read_csv(file_name)

    data_curr_month = data_curr_month.dropna(axis=0)

    data_curr_month.insert(loc=0, column='return_bin', value=np.nan)

    data_curr_month.loc[data_curr_month['yield_rate']>0, 'return_bin'] = 0
    data_curr_month.loc[data_curr_month['yield_rate']<=0, 'return_bin'] = 1

    if i_month == para.month_in_sample[0]:
        data_in_sample = data_curr_month
    else:
        data_in_sample = pd.concat([data_in_sample, data_curr_month])
        # data_in_sample = data_in_sample.append(data_curr_month)

data_in_sample

,return_bin,order_book_id,board_type,sector_code,month,date,yield_rate,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,...,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,0.0,000858.XSHE,MainBoard,ConsumerStaples,0,2017-01-03,0.104297,1.613199,-1.345188,2.151185,...,-1.308483,-1.185857,-0.554228,0.113686,-0.397188,0.063279,-1.006327,-1.155349,0.318017,0.464290
1,1.0,000858.XSHE,MainBoard,ConsumerStaples,1,2017-01-04,-0.054569,1.451644,-1.314485,1.982199,...,0.047494,-1.185857,0.016854,0.266345,-0.330853,0.056705,-1.008537,-1.156342,0.518065,0.497984
2,1.0,000858.XSHE,MainBoard,ConsumerStaples,2,2017-01-05,-0.002193,1.447873,-1.313742,1.978255,...,-0.071726,-0.484215,0.036787,0.213031,-0.294659,-0.036510,-1.010914,-1.157434,-0.058321,0.036318
3,0.0,000858.XSHE,MainBoard,ConsumerStaples,3,2017-01-06,0.072360,1.426626,-1.309533,1.956031,...,0.199560,0.217427,0.130619,0.206751,-0.252249,-0.080471,-1.020724,-1.160066,0.091241,0.318061
4,0.0,000858.XSHE,MainBoard,ConsumerStaples,4,2017-01-09,0.162670,1.383505,-1.300867,1.910927,...,1.404749,0.919069,0.295281,0.256981,-0.210209,-0.105825,-1.024128,-1.161755,0.302957,0.341891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,0.0,000858.XSHE,MainBoard,ConsumerStaples,1192,2021-11-29,0.099502,-0.660889,0.607266,-0.775233,...,1.354121,-0.484215,0.758131,-0.643120,-0.002117,-0.384084,0.372785,0.047029,0.885312,-0.080804
1190,0.0,000858.XSHE,MainBoard,ConsumerStaples,1193,2021-11-30,0.432198,-0.586699,0.523095,-0.714540,...,0.221318,-0.484215,0.180853,-0.661956,-0.012897,-0.377025,0.320287,0.043918,0.710295,-0.048422
1191,1.0,000858.XSHE,MainBoard,ConsumerStaples,1194,2021-12-01,-0.139106,-0.615454,0.555113,-0.738063,...,0.427977,0.217427,0.290921,-0.652498,-0.052859,-0.374602,0.224473,0.045263,0.535268,0.158648
1192,1.0,000858.XSHE,MainBoard,ConsumerStaples,1195,2021-12-02,-0.233235,-0.639528,0.582503,-0.757757,...,0.846222,0.217427,0.393590,-0.644533,-0.089828,-0.376770,0.250506,0.039398,0.813216,0.478872


In [28]:
X_in_sample = data_in_sample.loc[:, para.feature_column_start_name: para.feature_column_end_name]
y_in_sample = data_in_sample.loc[:, 'return_bin']

from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False) # True, random_state=para.seed)
# X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False)

X_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,1.613199,-1.345188,2.151185,-0.305592,-0.284820,-0.138772,-1.308483,-1.185857,-0.554228,0.113686,-0.397188,0.063279,-1.006327,-1.155349,0.318017,0.464290
1,1.451644,-1.314485,1.982199,-0.280020,-0.287093,-0.041630,0.047494,-1.185857,0.016854,0.266345,-0.330853,0.056705,-1.008537,-1.156342,0.518065,0.497984
2,1.447873,-1.313742,1.978255,-0.259030,-0.284470,0.023240,-0.071726,-0.484215,0.036787,0.213031,-0.294659,-0.036510,-1.010914,-1.157434,-0.058321,0.036318
3,1.426626,-1.309533,1.956031,-0.239357,-0.278210,0.071443,0.199560,0.217427,0.130619,0.206751,-0.252249,-0.080471,-1.020724,-1.160066,0.091241,0.318061
4,1.383505,-1.300867,1.910927,-0.217711,-0.268622,0.115534,1.404749,0.919069,0.295281,0.256981,-0.210209,-0.105825,-1.024128,-1.161755,0.302957,0.341891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,-1.420760,1.985605,-1.393367,2.181178,2.083823,0.729468,0.236031,-1.185857,0.513292,-1.439607,-1.324918,-1.103746,2.616838,1.235575,0.251596,0.387139
1070,-1.430035,2.009382,-1.400974,2.088991,2.109055,0.322828,0.282766,-0.484215,0.503776,-1.443787,-1.354600,-1.089824,2.539812,1.296430,0.187891,0.305864
1071,-1.438794,2.032084,-1.408158,2.020497,2.114750,0.063940,0.886041,-0.484215,0.473284,-1.456694,-1.410611,-1.089679,2.360004,1.304632,0.451239,0.312812
1072,-1.426380,1.999978,-1.397976,1.898167,2.093423,-0.294100,0.296676,-1.185857,0.211625,-1.445519,-1.478042,-1.091754,1.891235,1.288069,0.270633,0.115321


In [29]:
y_train

0       0.0
1       1.0
2       1.0
3       0.0
4       0.0
       ... 
1069    1.0
1070    1.0
1071    1.0
1072    1.0
1073    1.0
Name: return_bin, Length: 1074, dtype: float64

In [30]:
data_train = pd.concat([X_train, y_train], axis=1)
data_cv = pd.concat([X_cv, y_cv], axis=1)
data_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR,return_bin
0,1.613199,-1.345188,2.151185,-0.305592,-0.284820,-0.138772,-1.308483,-1.185857,-0.554228,0.113686,-0.397188,0.063279,-1.006327,-1.155349,0.318017,0.464290,0.0
1,1.451644,-1.314485,1.982199,-0.280020,-0.287093,-0.041630,0.047494,-1.185857,0.016854,0.266345,-0.330853,0.056705,-1.008537,-1.156342,0.518065,0.497984,1.0
2,1.447873,-1.313742,1.978255,-0.259030,-0.284470,0.023240,-0.071726,-0.484215,0.036787,0.213031,-0.294659,-0.036510,-1.010914,-1.157434,-0.058321,0.036318,1.0
3,1.426626,-1.309533,1.956031,-0.239357,-0.278210,0.071443,0.199560,0.217427,0.130619,0.206751,-0.252249,-0.080471,-1.020724,-1.160066,0.091241,0.318061,0.0
4,1.383505,-1.300867,1.910927,-0.217711,-0.268622,0.115534,1.404749,0.919069,0.295281,0.256981,-0.210209,-0.105825,-1.024128,-1.161755,0.302957,0.341891,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,-1.420760,1.985605,-1.393367,2.181178,2.083823,0.729468,0.236031,-1.185857,0.513292,-1.439607,-1.324918,-1.103746,2.616838,1.235575,0.251596,0.387139,1.0
1070,-1.430035,2.009382,-1.400974,2.088991,2.109055,0.322828,0.282766,-0.484215,0.503776,-1.443787,-1.354600,-1.089824,2.539812,1.296430,0.187891,0.305864,1.0
1071,-1.438794,2.032084,-1.408158,2.020497,2.114750,0.063940,0.886041,-0.484215,0.473284,-1.456694,-1.410611,-1.089679,2.360004,1.304632,0.451239,0.312812,1.0
1072,-1.426380,1.999978,-1.397976,1.898167,2.093423,-0.294100,0.296676,-1.185857,0.211625,-1.445519,-1.478042,-1.091754,1.891235,1.288069,0.270633,0.115321,1.0


In [31]:
from torch.utils.data import TensorDataset
import torch

X_train_ndarray = X_train.values
y_train_ndarray = y_train.values

train_dataset = TensorDataset(torch.from_numpy(X_train_ndarray).type(torch.FloatTensor), torch.from_numpy(y_train_ndarray).type(torch.LongTensor))

# for X_train_temp, y_train_temp in train_dataset:
#     print(X_train_temp, y_train_temp)
#     print(X_train_temp.dtype, y_train_temp.dtype)

In [32]:
X_cv_ndarray = X_cv.values
y_cv_ndarray = y_cv.values

cv_dataset = TensorDataset(torch.from_numpy(X_cv_ndarray).type(torch.FloatTensor), torch.from_numpy(y_cv.values).type(torch.LongTensor))

# for X_cv_temp, y_cv_temp in cv_dataset:
#     print(X_cv_temp, y_cv_temp)
#     print(X_cv_temp.dtype, y_cv_temp.dtype)

In [33]:
from torch.utils.data import DataLoader


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

cv_dataloader = DataLoader(
    dataset=cv_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

### 构建测试集

In [34]:
# data_test = None
# for i_month in para.month_test:
#
#     file_name = para.data_path + '/' + str(i_month) + '.csv'
#     data_curr_month = pd.read_csv(file_name)
#
#     data_curr_month = data_curr_month.dropna(axis=0)
#
#     data_curr_month = label_data(data=data_curr_month, percent_select=para.percent_select)
#
#     if i_month == para.month_test[0]:
#         data_test = data_curr_month
#     else:
#         data_test = pd.concat([data_test, data_curr_month])
#         # data_test = data_test.append(data_curr_month)
#
# X_test = data_test.loc[:, para.feature_column_start_name: para.feature_column_end_name]
# y_test = data_test.loc[:, 'return_bin']

In [35]:
# from torch.utils.data import TensorDataset
# import torch
#
#
# X_test_ndarray = X_test.values
# y_test_ndarray = y_test.values
#
# test_dataset = TensorDataset(torch.from_numpy(X_test_ndarray).type(torch.FloatTensor), torch.from_numpy(y_test_ndarray).type(torch.LongTensor))

In [36]:
# from torch.utils.data import DataLoader
#
#
# test_dataloader = DataLoader(
#     dataset=test_dataset,
#     batch_size=para.batch_size,
#     shuffle=True,
# )

### 构建神经网络

In [37]:
from my_utils.model_class import MLP5

model = MLP5(in_nums=len(X_train.columns), out_nums=para.classification, drop_p=para.drop)
# to device
model = model.to(device=para.device)
print(model)

MLP5(
  (linear_stack): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=8, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=8, bias=True)
    (7): Dropout(p=0.5, inplace=False)
    (8): ReLU()
    (9): Linear(in_features=8, out_features=8, bias=True)
    (10): Dropout(p=0.5, inplace=False)
    (11): ReLU()
    (12): Linear(in_features=8, out_features=2, bias=True)
  )
)


### 训练与测试

In [38]:
import torchmetrics


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0

    # initialize metric
    train_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    for batch, (X, y) in enumerate(dataloader):

        # to device
        X = X.to(device=para.device)
        y = y.to(device=para.device)

        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        train_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # metric on current batch
        train_precision(pred.argmax(1), y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    train_loss /= num_batches
    correct /= size
    print(f"Train Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = train_precision.compute()
    print("Precision of every train dataset class: ", total_precision)
    print()

    return correct, train_loss, total_precision


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # initialize metric
    test_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    with torch.no_grad():
        for X, y in dataloader:

            # to device
            X = X.to(device=para.device)
            y = y.to(device=para.device)

            # compute prediction and loss
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # metric on current batch
            test_precision(pred.argmax(1), y)


    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = test_precision.compute()
    print("Precision of every test dataset class: ", total_precision)
    print()

    return correct, test_loss, total_precision

In [39]:
def select_df_to_dataloader(df: pd.DataFrame, select: int) -> DataLoader:

    df = df[df['return_bin'] == select]

    df_dataset = TensorDataset(
        torch.from_numpy(df.loc[:, para.feature_column_start_name: para.feature_column_end_name].values).type(torch.FloatTensor),
        torch.from_numpy(df.loc[:, 'return_bin'].values).type(torch.LongTensor))

    df_dataloader = DataLoader(
        dataset=df_dataset,
        batch_size=para.batch_size,
        shuffle=True,
    )

    return df_dataloader

In [40]:
# temp2_dataloader = select_df_to_dataloader(df=data_cv, select=2)
temp1_dataloader = select_df_to_dataloader(df=data_cv, select=1)
temp0_dataloader = select_df_to_dataloader(df=data_cv, select=0)

In [41]:
from torch import nn
import time
from torch.utils.tensorboard import SummaryWriter

# 计时
time_start = time.time()

# writer = SummaryWriter(para.tensor_board_log_dir)
writer = SummaryWriter()

# 损失函数
loss_fn = nn.CrossEntropyLoss()
# to device
loss_fn = loss_fn.to(device=para.device)

# 优化器
optimizer = torch.optim.Adam(model.parameters(), lr=para.lr)

epochs = para.epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    model.train()
    accuracy_train, loss_train, precision_train = train_loop(train_dataloader, model, loss_fn, optimizer)
    model.eval()
    accuracy_cv, loss_cv, precision_cv = test_loop(cv_dataloader, model, loss_fn)

    # accuracy2 = test_loop(temp2_dataloader, model, loss_fn)
    # print('#')
    # accuracy1 = test_loop(temp1_dataloader, model, loss_fn)
    # accuracy0 = test_loop(temp0_dataloader, model, loss_fn)

    # 写入 tensorboard
    if para.classification == 2:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1]},
                           global_step=t)

    elif para.classification == 3:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2]},
                           global_step=t)

    elif para.classification == 5:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2],
                               'precision3': precision_cv[3],
                               'precision4': precision_cv[4]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2],
                               'precision3': precision_train[3],
                               'precision4': precision_train[4]},
                           global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/cv',
                       tag_scalar_dict={
                           'loss': loss_cv},
                       global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/train',
                       tag_scalar_dict={
                           'loss': loss_train},
                       global_step=t)
    writer.flush()

    time_end = time.time()
    print('Time cost = %fs' % (time_end - time_start))
    print()

writer.close()

print("Done!")

Epoch 1
-------------------------------
Train Error: 
    Accuracy: 48.2%, Avg loss: 0.701871 

Precision of every train dataset class:  tensor([0.4820, 0.4824], device='cuda:0')

Test Error: 
    Accuracy: 65.0%, Avg loss: 0.690142 

Precision of every test dataset class:  tensor([0.0000, 0.6667], device='cuda:0')

Time cost = 1.055179s

Epoch 2
-------------------------------
Train Error: 
    Accuracy: 49.2%, Avg loss: 0.694274 

Precision of every train dataset class:  tensor([0.4982, 0.4843], device='cuda:0')

Test Error: 
    Accuracy: 41.7%, Avg loss: 0.698361 

Precision of every test dataset class:  tensor([0.3495, 0.8235], device='cuda:0')

Time cost = 1.924853s

Epoch 3
-------------------------------
Train Error: 
    Accuracy: 49.1%, Avg loss: 0.696457 

Precision of every train dataset class:  tensor([0.4974, 0.4826], device='cuda:0')

Test Error: 
    Accuracy: 40.0%, Avg loss: 0.699323 

Precision of every test dataset class:  tensor([0.3429, 0.8000], device='cuda:0')



## 保存模型

In [42]:
torch.save(model.state_dict(), para.save_model_path)

print('Finish save model!')

Finish save model!


## captum

In [43]:
# # captum
# from captum.attr import IntegratedGradients
#
# ig = IntegratedGradients(model)

In [44]:
# temp = cv_dataloader.dataset.tensors[0]
# temp.requires_grad_()
# attr, delta = ig.attribute(temp,target=1, return_convergence_delta=True)
# attr = attr.detach().numpy()

In [45]:
# # Helper method to print importances and visualize distribution
# def visualize_importances(feature_names, importances, title="Average Feature Importances", plot=True, axis_title="Features"):
#     print(title)
#     for i in range(len(feature_names)):
#         print(feature_names[i], ": ", '%.3f'%(importances[i]))
#     y_pos = (np.arange(len(feature_names)))
#     if plot:
#         plt.figure(figsize=(20,6))
#         plt.barh(y_pos, importances, align='center')
#         plt.yticks(y_pos, feature_names)
#         plt.ylabel(axis_title)
#         plt.grid(axis='y')
#         plt.title(title)
# visualize_importances(feature_names=X_cv.columns.values.tolist(), importances=np.mean(attr, axis=0))

In [46]:
# X_cv.columns.values.tolist()

In [47]:
# loss = nn.CrossEntropyLoss()
# input = torch.Tensor(
#     [[-0.0441,  0.0773],
#     [-0.0781, -0.1772],
#     [-0.1319, -0.0432],
#     [-0.0714, -0.1261],
#     [-0.0806, -0.1370],
#     [-0.1730, -0.1472],
#     [-0.0350, -0.0507],
#     [-0.1149, -0.2248]])
# # input = input.reshape(-1,4)
# target = torch.Tensor([0, 1, 1, 0, 0, 0, 0, 0]).type(torch.LongTensor)
# print(input.dtype)
# print(target.dtype)
# output = loss(input, target)
# print(input, target, output)

In [48]:
# # Example of target with class indices
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.Tensor([1,4,1]).type(torch.LongTensor)
# output = loss(input, target)
# print(input,target,output)

In [49]:
# loss = nn.BCEWithLogitsLoss()
# input = torch.Tensor([0.5, 0.4, 0.3])
# target = torch.Tensor([0])
# output = loss(input, target)
# print(input, target, output)